## TRAIN DATA

In [ ]:
#Train data



import cv2
import sys
import os

size = 4

webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
count=0
c=0
num = int(input("Enter the no. of user u wanna add : "))
user=1
if not os.path.exists('user'):
    os.makedirs('user')
i=1    
while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1) #Flip to act as a mirror
    gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    # Resize the image to speed up detection
    mini = cv2.resize(im, (int(im.shape[1] / size),int(im.shape[0] / size)))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)
    
    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        cv2.rectangle(gray, (x, y), (x + w, y + h),(0,255,0),thickness=4)
        #Save just the rectangle faces in SubRecFaces
        sub_face = gray[y:y+h, x:x+w]
        count+=1
        c+=1
        FaceFileName = "./user/_" + str(i) + "_" + str(count) + ".jpg"
        cv2.imwrite(FaceFileName, sub_face)

    # Show the image
   # cv2.imshow('capturing images..',   im)
    cv2.putText(gray,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX , 1, (0,255,0),2)
    cv2.imshow('Face Cropper',gray)
    
    if(c%100)==0 and c!=num*100 and c!=0:
        print("Place the new user Signature")
        i+=1
        count=0
        input() #press enter after the new person has arrived.
    key = cv2.waitKey(100)
    
    # if Esc key is press then break out of the loop 
    if key == 27 or c==num*100:#The Esc key
        break 
webcam.release()        
cv2.destroyAllWindows() 

##  Model Training and FACE IDENTIFICATION STEP

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

#Get the training data we previously made
data_path = './user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

#Create arrays for training data and labels

Training_Data, Labels = [], []

#Open training images in our datapath
#Create a numpy array for training data
for i,files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
    
#Create a numpy array for both training and data lables
#Labels = np.asarray(images, dtype=np.int32)

#Initialize face recognizer
model = cv2.face_LBPHFaceRecognizer.create()

#Let's train our model
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained successsfully")

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#FACE IDENTIFICATIN STEP

def face_detector(img,size=0.5):
    
    #convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(frame)
    if faces is ():
        return img,[]
    
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200,200))
    return img,roi

#Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    
    image, face = face_detector(frame)
    
    try: 
        
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)

                #Pass face to prediction model
                #"results" comprises of a tuple containing the label and the success value
        results = model.predict(face)
        im=cv2.flip(image,1)

        if results[1] < 500:
            success = int( 100 * (1- (results[1])/400) )
            display_string = str(success) + '% success Authorised user'

            cv2.putText(im, display_string, (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

        if success >85:
            cv2.putText(im, "Unlocked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', im)

        else:
            cv2.putText(im, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', im)
            
    except:  
        cv2.putText(im,"No face found",(220,850),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        cv2.imshow('Face Recognition', im)
        pass

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()